# Problem Set 3
### 1. Select only male heads of household who are between 25 and 60 years of age and earn wages > \$ 7

In [35]:
# import packages
import pandas as pd
import numpy as np
import scipy.optimize as opt
import time

In [36]:
# Read in the data
ps3_data = pd.read_stata('PS3_data.dta')
ps3_data.head(n=5)

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,redpregovinc,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce
0,1,1967,1,Head,1200.0,2000.0,NaN,NaN,0,NaN,...,5614.0,1.0,2.0,52.0,46.0,1.0,2.0,8.0,8.0,0.0
1,2,1967,2,Head,0.0,0.0,NaN,NaN,0,NaN,...,0.0,1.0,2.0,56.0,57.0,1.0,2.0,3.0,3.0,0.0
2,3,1967,3,Head,0.0,0.0,NaN,NaN,0,NaN,...,0.0,1.0,2.0,77.0,64.0,1.0,2.0,NaN,3.0,0.0
3,4,1967,4,Head,1560.0,0.0,NaN,NaN,6,1.0,...,3280.0,1.0,2.0,45.0,44.0,1.0,2.0,8.0,5.0,0.0
4,5,1967,5,Head,2500.0,2000.0,NaN,NaN,3,1.0,...,7900.0,1.0,2.0,24.0,22.0,1.0,2.0,10.0,9.0,0.0


In [37]:
# Drop missing data points for income and hours and where hours =0
ps3_data = ps3_data[(ps3_data.hlabinc.isnull() != True ) & (ps3_data.hannhrs.isnull() != True ) & (ps3_data.hannhrs > 0)]

In [38]:
# Drop missing data points for education and age and race
ps3_data = ps3_data[(ps3_data.hyrsed.isnull() != True) & (ps3_data.age.isnull() != True) & (ps3_data.hrace.isnull() != True)]

In [39]:
# Define wage per hour
ps3_data['wageperhour'] = ps3_data['hlabinc'] / ps3_data['hannhrs'] 

# Define log of wage per hour
ps3_data['lnwage'] = np.log(ps3_data['wageperhour'])

In [40]:
# Pick only male heads of household who are between 25 and 60 years old and earn wages > $7/hr
ps3_data = ps3_data[(ps3_data.hsex ==1 ) & (ps3_data.age >= 25) & (ps3_data.age <= 60) & (ps3_data.wageperhour > 7)]

### 2. Create indicator and continuous variables as necessary.

In [41]:
# Create three dummy variables each for black, hispanic and other race. 1 means true, 0 means false
ps3_data['Black'] = ps3_data.loc[:,'hrace'].apply(lambda x : 1 if x == 2 else 0)
ps3_data['Hispanic'] = ps3_data.loc[:,'hrace'].apply(lambda x : 1 if x == 5 else 0)
ps3_data['OtherRace'] = ps3_data.loc[:,'hrace'].apply(lambda x : 1 if x == 3 or x == 4 or x == 6 or x == 7 else 0)

### 3. Estimate the following model via a Maximum Likelihood Estimator separately for t = 1971, 1980, 1990, 2000:

In [47]:
# Defining the constant term and entering in year variable. I changed this variable for 1971, 1980, 1990 and 2000 each time
# and started running to code from the first line. 
ps3_data['constant'] = 1
ps3_data = ps3_data[(ps3_data['year'] == 2000)]

In [45]:
# Under the nomrally distributed error terms assumption, the log-likelihood function is defined as below:
def loglike(params, x, y):
    
    beta = np.array([[params[0]], [params[1]], [params[2]], [params[3]], [params[4]]], dtype = np.float64)
    
    x = np.array([ps3_data['constant'], ps3_data['hyrsed'], ps3_data['age'], 
                  ps3_data['Black'], ps3_data['OtherRace']]).T
    
    y = np.array([ps3_data['lnwage']]).T
      
    sigma = 2
    
    params = [alpha, beta1, beta2, beta3, beta5]
    
    log_like = (- (-(1 / 2) * len(ps3_data.index) * np.log(2 * (np.pi)) - len(ps3_data.index) * np.log(sigma) - 
                  (1 / 2) * np.dot((y - np.dot(x, beta)).T, y - np.dot(x, beta)) / ((sigma) ** 2))).item((0,0))
    
    return log_like

In [46]:
# Initial guesses for the parameter values
alpha, beta1, beta2, beta3, beta5 = 0.1, -0.1, 0.2, -0.1, 0.5
params = [alpha, beta1, beta2, beta3, beta5]
params_initial = (0.3, -0.5, 0.6, -0.3, 0.8)


# Seting boundaries for the parameters
bnds = ([-np.inf, -np.inf, -np.inf, -np.inf, -np.inf],
        [np.inf, np.inf, np.inf, np.inf, np.inf])

# Employing minimize function to minimize the minus log-likelihood function
mle_results = opt.minimize(loglike, params_initial, args=(ps3_data,params),
                           method='Nelder-Mead', tol = 1e-12, options={'maxiter': 5000})

mle_results

 final_simplex: (array([[ 1.16169058,  0.10915437,  0.01099353, -0.24604485, -0.06073224],
       [ 1.16169058,  0.10915437,  0.01099353, -0.24604485, -0.06073224],
       [ 1.16169058,  0.10915437,  0.01099353, -0.24604485, -0.06073224],
       [ 1.16169058,  0.10915437,  0.01099353, -0.24604485, -0.06073224],
       [ 1.16169058,  0.10915437,  0.01099353, -0.24604485, -0.06073224],
       [ 1.16169058,  0.10915437,  0.01099353, -0.24604485, -0.06073224]]), array([ 4253.06800934,  4253.06800934,  4253.06800934,  4253.06800934,
        4253.06800934,  4253.06800934]))
           fun: 4253.0680093370647
       message: 'Optimization terminated successfully.'
          nfev: 1211
           nit: 686
        status: 0
       success: True
             x: array([ 1.16169058,  0.10915437,  0.01099353, -0.24604485, -0.06073224])

### 4. Interpret the coefficient beta 1. How do the returns to education change over time in these data?

#### This coefficient is the returns on education holding all the other variables constant. If one gets one more year of education, the percentage change in his/her wage will be 100*Beta1.

#### For year 1971, beta1=0.0669 so, if one gets one more year of education, his/her wage will increase by 6.69%.

#### For year 1980, beta1= 0.0676 so, if one gets one more year of education, his/her wage will increase by 6.76%.

#### For year 1990, beta1= 0.0976 so, if one gets one more year of education, his/her wage will increase by 9.76%.

#### For year 2000, beta1= 0.1092 so, if one gets one more year of education, his/her wage will increase by 10.92%.

#### Therefore, the returns on education increases over time. 